In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import os

# Caminho para o diretório onde estão os arquivos
caminho = "../Main/Instances Matriz"  # Atualizado conforme solicitado

def carregar_dataframes(caminho):
    """
    Carrega todos os arquivos CSV do diretório especificado.

    Args:
        caminho (str): Caminho para o diretório contendo os arquivos CSV.

    Returns:
        dict: Dicionário com nomes dos arquivos como chaves e DataFrames como valores.
    """
    arquivos = [os.path.join(caminho, f) for f in os.listdir(caminho) if f.endswith(".csv")]
    if not arquivos:
        raise ValueError("Nenhum arquivo CSV encontrado no diretório especificado.")
    
    dataframes = {}
    for arquivo in arquivos:
        nome = os.path.basename(arquivo).split(".")[0]  # Nome do arquivo sem extensão
        try:
            df = pd.read_csv(arquivo)
            dataframes[nome] = df
        except Exception as e:
            print(f"Erro ao carregar {arquivo}: {e}")
    return dataframes

def coletar_todas_colunas(dataframes):
    """
    Coleta todas as colunas únicas de todos os DataFrames.

    Args:
        dataframes (dict): Dicionário de DataFrames.

    Returns:
        list: Lista ordenada de todas as colunas únicas.
    """
    todas_colunas = set()
    for df in dataframes.values():
        todas_colunas.update(df.columns)
    return sorted(todas_colunas)

def padronizar_colunas(df, todas_colunas):
    """
    Padroniza as colunas de um DataFrame, adicionando colunas ausentes com valor 0.0.

    Args:
        df (DataFrame): DataFrame a ser padronizado.
        todas_colunas (list): Lista de todas as colunas únicas.

    Returns:
        DataFrame: DataFrame padronizado.
    """
    colunas_ausentes = [col for col in todas_colunas if col not in df.columns]
    if colunas_ausentes:
        colunas_ausentes_df = pd.DataFrame(0.0, index=df.index, columns=colunas_ausentes)
        df = pd.concat([df, colunas_ausentes_df], axis=1)
    return df[todas_colunas]

def carregar_e_padronizar_dataframes(caminho):
    """
    Carrega e padroniza todos os DataFrames do diretório especificado.

    Args:
        caminho (str): Caminho para o diretório contendo os arquivos CSV.

    Returns:
        tuple: Dicionário de DataFrames padronizados e a lista de todas as colunas.
    """
    dataframes = carregar_dataframes(caminho)
    todas_colunas = coletar_todas_colunas(dataframes)
    dataframes_padronizados = {nome: padronizar_colunas(df, todas_colunas) for nome, df in dataframes.items()}
    return dataframes_padronizados, todas_colunas

def preparar_dados(dataframes_padronizados, todas_colunas, num_train=30):
    """
    Separa os DataFrames em conjuntos de treinamento e teste.

    Args:
        dataframes_padronizados (dict): Dicionário de DataFrames padronizados.
        todas_colunas (list): Lista de todas as colunas únicas.
        num_train (int): Número de DataFrames para treinamento.

    Returns:
        tuple: Dados de treinamento, rótulos de treinamento, dados de teste e colunas de vértices.
    """
    dataframes_items = list(dataframes_padronizados.items())
    train_data = dict(dataframes_items[:num_train])
    test_data = dict(dataframes_items[num_train:])
    
    if not train_data:
        raise ValueError("Nenhum DataFrame disponível para treinamento.")
    if not test_data:
        raise ValueError("Nenhum DataFrame disponível para teste.")
    
    # Identificar colunas de vértices (assumindo que 'Conjuntos' é a coluna alvo)
    colunas_vertices = [col for col in todas_colunas if col != "Conjuntos"]
    
    # Concatenar DataFrames de treinamento
    train_df = pd.concat(train_data.values(), ignore_index=True)
    X_train = train_df[colunas_vertices].copy()
    y_train = train_df["Conjuntos"].astype(int)
    
    return X_train, y_train, test_data, colunas_vertices

def treinar_modelos(X_train, y_train, usar_pca=True):
    """
    Normaliza, balanceia, aplica PCA e treina múltiplos modelos com otimização de hiperparâmetros.

    Args:
        X_train (DataFrame): Dados de treinamento.
        y_train (Series): Rótulos de treinamento.
        usar_pca (bool): Se deve aplicar PCA.

    Returns:
        dict: Dicionário contendo scaler, PCA (se usado) e modelos treinados.
    """
    # Normalizar os dados
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Tratar desbalanceamento com SMOTE
    smote = SMOTE(random_state=42)
    X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)
    
    # Aplicar PCA opcionalmente
    if usar_pca:
        pca = PCA(n_components=0.95, random_state=42)
        X_train_bal = pca.fit_transform(X_train_bal)
    else:
        pca = None
    
    # Ajustar pesos das classes
    classes = np.array([1, 2])
    pesos = compute_class_weight('balanced', classes=classes, y=y_train_bal)
    class_weight_dict = {1: pesos[0], 2: pesos[1]}
    
    # Dicionário para armazenar os modelos treinados
    modelos_treinados = {}

    # Otimização para cada modelo
    modelos_com_parametros = {
        "RandomForest": (RandomForestClassifier(random_state=42, class_weight=class_weight_dict), {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_leaf': [1, 2, 4],
            'criterion': ['gini', 'entropy']
        }),
        "LogisticRegression": (LogisticRegression(random_state=42, class_weight=class_weight_dict, max_iter=1000), {
            'C': [0.01, 0.1, 1, 10]
        }),
        #"SVM": (SVC(probability=True, random_state=42), {
            #'C': [0.1, 1, 10],
            #'kernel': ['linear', 'poly', 'rbf'],
            #'gamma': ['scale', 'auto']
        #}),
        "KNN": (KNeighborsClassifier(), {
            'n_neighbors': [3, 5, 7],
            'metric': ['euclidean', 'manhattan', 'minkowski']
        }),
        "DecisionTree": (DecisionTreeClassifier(random_state=42, class_weight=class_weight_dict), {
            'max_depth': [None, 10, 20],
            'min_samples_leaf': [1, 2, 4],
            'criterion': ['gini', 'entropy']
        })
    }

    # Treinamento com otimização de hiperparâmetros
    for nome, (modelo, parametros) in modelos_com_parametros.items():
        print(f"Treinando e otimizando modelo: {nome}")
        grid_search = GridSearchCV(
            modelo, parametros, cv=3, scoring='f1', n_jobs=-1, verbose=2
        )
        grid_search.fit(X_train_bal, y_train_bal)
        modelos_treinados[nome] = grid_search.best_estimator_
        print(f"Melhores hiperparâmetros para {nome}: {grid_search.best_params_}")
    
    return scaler, pca, modelos_treinados

def avaliar_modelos(modelos_treinados, scaler, pca, test_data, todas_colunas, colunas_vertices):
    """
    Avalia múltiplos modelos nos dados de teste e coleta os vértices da classe 2.

    Args:
        modelos_treinados (dict): Dicionário de modelos treinados.
        scaler (StandardScaler): Objeto scaler treinado.
        pca (PCA): Objeto PCA treinado ou None.
        test_data (dict): Dicionário de DataFrames de teste.
        todas_colunas (list): Lista de todas as colunas únicas.
        colunas_vertices (list): Lista de colunas de vértices.

    Returns:
        dict: Dicionário de DataFrames contendo os resultados para cada modelo.
    """
    resultados_modelos = {nome: [] for nome in modelos_treinados.keys()}
    
    for nome_modelo, modelo in modelos_treinados.items():
        print(f"Avaliação do modelo: {nome_modelo}")
        resultados_detalhados = []
        for nome_instancia, df in test_data.items():
            # Preparar dados de teste
            X_test = df[colunas_vertices].copy()
            y_test = df["Conjuntos"].astype(int)
            
            # Normalizar e aplicar PCA opcionalmente
            X_test_scaled = scaler.transform(X_test)
            if pca:
                X_test_scaled = pca.transform(X_test_scaled)
            
            # Predição e obtenção de probabilidades
            y_pred = modelo.predict(X_test_scaled)
            y_proba = modelo.predict_proba(X_test_scaled)
            
            # Determinar o número de vértices da diversidade máxima pela instância
            # Assumindo o padrão 'SOM-a_24_n100_m10_matriz_adjacencia_caracteristicas'
            # onde 'm10' indica m = 10
            try:
                m_part = [part for part in nome_instancia.split("_") if part.startswith("m")][0]
                num_vertices_needed = int(m_part[1:])
            except (IndexError, ValueError):
                print(f"Erro ao extrair 'm' de {nome_instancia}. Definindo num_vertices_needed=0.")
                num_vertices_needed = 0
            
            # Garantir que num_vertices_needed não exceda o número total de vértices
            total_vertices = len(colunas_vertices)
            if num_vertices_needed > total_vertices:
                print(f"Warning: 'm' ({num_vertices_needed}) excede o número total de vértices ({total_vertices}). Ajustando 'm' para {total_vertices}.")
                num_vertices_needed = total_vertices
            
            # Ajustar as predições pelo ranqueamento corrigido
            y_pred_adjusted = rank_and_adjust_corrected(y_pred, y_proba, num_vertices_needed)
            
            # Verificação para garantir o número correto de classe 2
            num_class_2 = np.sum(y_pred_adjusted == 2)
            if num_class_2 != num_vertices_needed:
                print(f"Erro: Para {nome_instancia}, esperado {num_vertices_needed} vértices na classe 2, mas encontrado {num_class_2}.")
                # Opcionalmente, você pode decidir como lidar com esse caso
            
            # Relatório de métricas detalhado
            relatorio = classification_report(y_test, y_pred_adjusted, output_dict=True, zero_division=0)
            
            # Obter a lista de vértices na classe 2
            vertices_classe_2 = df.iloc[y_pred_adjusted == 2]
            vertices_lista = vertices_classe_2.index.tolist() if not vertices_classe_2.empty else []
            
            # Adicionar resultados para cada classe
            for classe in [1, 2]:
                if str(classe) in relatorio:
                    resultados_modelos[nome_modelo].append({
                        "Instância": nome_instancia,
                        "Classe": classe,
                        "Precisão": f"{relatorio[str(classe)]['precision']:.2f}" if not np.isnan(relatorio[str(classe)]['precision']) else "0.00",
                        "Recall": f"{relatorio[str(classe)]['recall']:.2f}" if not np.isnan(relatorio[str(classe)]['recall']) else "0.00",
                        "F1-Score": f"{relatorio[str(classe)]['f1-score']:.2f}" if not np.isnan(relatorio[str(classe)]['f1-score']) else "0.00",
                        "Suporte": relatorio[str(classe)]['support'],
                        "Vértices Classe 2": vertices_lista if classe == 2 else None,
                    })
        
        # Converter resultados detalhados para DataFrame
        resultados_detalhados_df = pd.DataFrame(resultados_modelos[nome_modelo])
        
        # Reorganizar colunas para melhor visualização
        colunas_order = ["Instância", "Classe", "Precisão", "Recall", "F1-Score", "Suporte", "Vértices Classe 2"]
        resultados_detalhados_df = resultados_detalhados_df[colunas_order]
        
        resultados_modelos[nome_modelo] = resultados_detalhados_df
    
    return resultados_modelos

def rank_and_adjust_corrected(y_pred, y_proba, num_vertices_needed):
    """
    Ajusta as predições para garantir que exatamente num_vertices_needed instâncias sejam classificadas como classe 2.
    Isso pode ser feito selecionando as instâncias com as maiores probabilidades de classe 2.

    Args:
        y_pred (np.array): Predições originais do modelo.
        y_proba (np.array): Probabilidades das classes.
        num_vertices_needed (int): Número de instâncias que devem ser classificadas como classe 2.

    Returns:
        np.array: Predições ajustadas.
    """
    if num_vertices_needed == 0:
        return y_pred  # Sem ajuste necessário
    
    # Obter as probabilidades de classe 2
    proba_class_2 = y_proba[:, 1]
    
    # Obter os índices ordenados pelas maiores probabilidades de classe 2
    sorted_indices = np.argsort(proba_class_2)[::-1]
    
    # Inicializar predições ajustadas com as predições originais
    y_pred_adjusted = y_pred.copy()
    
    # Definir as primeiras num_vertices_needed instâncias como classe 2
    y_pred_adjusted[sorted_indices[:num_vertices_needed]] = 2
    
    # Garantir que apenas num_vertices_needed instâncias sejam classe 2
    y_pred_adjusted[y_pred_adjusted == 2] = 1  # Temporariamente
    y_pred_adjusted[sorted_indices[:num_vertices_needed]] = 2
    
    return y_pred_adjusted

def salvar_resultados(resultados_modelos):
    """
    Salva os resultados de cada modelo em arquivos CSV separados.

    Args:
        resultados_modelos (dict): Dicionário de DataFrames contendo os resultados para cada modelo.
    """
    for nome_modelo, df in resultados_modelos.items():
        nome_arquivo = f"../Main/Resultados Modelos Matriz/resultados_treinamento_{nome_modelo}.csv"
        df.to_csv(nome_arquivo, index=False, float_format='%.2f')
        print(f"Resultados do modelo {nome_modelo} salvos em '{nome_arquivo}'.")

def salvar_resultados_consolidados(resultados_modelos, caminho_saida="../Main/Resultados Modelos Matriz/resultados_consolidados.csv"):
    """
    Salva os resultados de todos os modelos em um único arquivo CSV com uma coluna indicando o modelo.

    Args:
        resultados_modelos (dict): Dicionário de DataFrames contendo os resultados para cada modelo.
        caminho_saida (str): Caminho para salvar o arquivo CSV consolidado.
    """
    df_consolidado = []
    for nome_modelo, df in resultados_modelos.items():
        df_temp = df.copy()
        df_temp["Modelo"] = nome_modelo
        df_consolidado.append(df_temp)
    
    df_consolidado = pd.concat(df_consolidado, ignore_index=True)
    
    # Reorganizar colunas
    colunas_order = ["Modelo", "Instância", "Classe", "Precisão", "Recall", "F1-Score", "Suporte", "Vértices Classe 2"]
    df_consolidado = df_consolidado[colunas_order]
    
    # Salvar
    df_consolidado.to_csv(caminho_saida, index=False, float_format='%.2f')
    print(f"Resultados consolidados salvos em '{caminho_saida}'.")

# Função principal para orquestrar o fluxo
def main():
    # Carregar e padronizar DataFrames
    dataframes_padronizados, todas_colunas = carregar_e_padronizar_dataframes(caminho)
    
    # Preparar dados de treinamento e teste
    # Ajuste num_train conforme necessário (30 no seu exemplo)
    X_train, y_train, test_data, colunas_vertices = preparar_dados(dataframes_padronizados, todas_colunas, num_train=150)
    
    # Treinar os modelos
    scaler, pca, modelos_treinados = treinar_modelos(X_train, y_train, usar_pca=True)
    
    # Avaliar os modelos
    resultados_modelos = avaliar_modelos(modelos_treinados, scaler, pca, test_data, todas_colunas, colunas_vertices)
    
    # Salvar resultados individuais
    salvar_resultados(resultados_modelos)
    
    # Opcional: Salvar resultados consolidados
    salvar_resultados_consolidados(resultados_modelos, caminho_saida="../Main/Resultados Modelos Matriz/resultados_consolidados.csv")
    
    # Exibir as primeiras linhas dos DataFrames de resultados
    for nome_modelo, df in resultados_modelos.items():
        print(f"\nResultados do modelo: {nome_modelo}")
        print(df.head())

# Executar a função principal
if __name__ == "__main__":
    main()


Treinando e otimizando modelo: RandomForest
Fitting 3 folds for each of 54 candidates, totalling 162 fits
Melhores hiperparâmetros para RandomForest: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}
Treinando e otimizando modelo: LogisticRegression
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Melhores hiperparâmetros para LogisticRegression: {'C': 0.01}
Treinando e otimizando modelo: KNN
Fitting 3 folds for each of 9 candidates, totalling 27 fits
Melhores hiperparâmetros para KNN: {'metric': 'euclidean', 'n_neighbors': 7}
Treinando e otimizando modelo: DecisionTree
Fitting 3 folds for each of 18 candidates, totalling 54 fits
Melhores hiperparâmetros para DecisionTree: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1}
Avaliação do modelo: RandomForest
Avaliação do modelo: LogisticRegression
Avaliação do modelo: KNN
Avaliação do modelo: DecisionTree
Resultados do modelo RandomForest salvos em '../Main/Resultados Modelos

In [2]:
import os
import pandas as pd
import numpy as np
import ast

def main():
    # Caminhos
    csv_input_path = "../Main/Resultados Modelos Matriz/resultados_consolidados.csv"  # Caminho para o CSV de entrada
    txt_folder = "../Instances"  # Caminho para a pasta contendo os arquivos .txt
    csv_output_path = "../Main/Resultados Modelos Matriz/resultados_processados.csv"  # Caminho para o CSV de saída

    # Verificações Iniciais
    if not os.path.exists(csv_input_path):
        raise FileNotFoundError(f"O arquivo CSV de entrada '{csv_input_path}' não foi encontrado.")
    
    if not os.path.isdir(txt_folder):
        raise NotADirectoryError(f"A pasta '{txt_folder}' não foi encontrada ou não é um diretório.")
    
    # Ler o arquivo CSV de entrada
    df = pd.read_csv(csv_input_path)

    # Verificar se as colunas necessárias existem
    colunas_necessarias = ["Instância", "Modelo", "Classe", "Vértices Classe 2"]
    for coluna in colunas_necessarias:
        if coluna not in df.columns:
            raise ValueError(f"A coluna '{coluna}' não foi encontrada no CSV de entrada.")

    # Filtrar apenas as linhas onde a Classe é 2
    df_classe2 = df[df["Classe"] == 2].copy()

    # Função para parsear a string de lista de vértices
    def parse_vertices(vertices_str):
        """
        Converte uma string representando uma lista de vértices em uma lista real.
        
        Args:
            vertices_str (str): String no formato "[v1, v2, ...]".
        
        Returns:
            list: Lista de vértices como strings.
        """
        try:
            # Utilizar ast.literal_eval para converter a string em uma lista
            vertices = ast.literal_eval(vertices_str)
            # Garantir que todos os vértices sejam strings
            return [str(v) for v in vertices]
        except (ValueError, SyntaxError) as e:
            print(f"Erro ao parsear vértices: '{vertices_str}' - {e}")
            return []

    # Função para extrair o nome base do arquivo .txt
    def extrair_nome_base(instancia):
        """
        Extrai a parte do nome da instância que corresponde ao nome do arquivo .txt.
        
        Exemplo:
            'GKD-a_4_n10_m2_matriz_adjacencia_caracteristicas' -> 'GKD-a_4_n10_m2'
        
        Args:
            instancia (str): Nome completo da instância.
        
        Returns:
            str: Nome base correspondente ao arquivo .txt.
        """
        try:
            # Supondo que a parte relevante termina antes de '_matriz_adjacencia_caracteristicas'
            nome_base = instancia.split('_matriz_adjacencia_caracteristicas')[0]
            return nome_base
        except IndexError:
            print(f"Erro ao extrair nome base da instância '{instancia}'.")
            return instancia  # Retorna o nome completo se o padrão não for encontrado

    # Inicializar listas para armazenar os resultados
    instancias_final = []
    modelos_final = []
    diversidade_maxima_final = []
    vertices_selecionados_final = []

    # Iterar sobre as linhas filtradas (Classe=2)
    for index, row in df_classe2.iterrows():
        instancia_completa = row["Instância"]
        modelo = row["Modelo"]
        vertices_str = row["Vértices Classe 2"]
        vertices = parse_vertices(vertices_str)
        
        # Extrair o nome base para construir o nome do arquivo .txt
        nome_base = extrair_nome_base(instancia_completa)
        
        if not vertices:
            # Se não houver vértices, atribuir 0.00 como diversidade máxima
            instancias_final.append(nome_base)
            modelos_final.append(modelo)
            diversidade_maxima_final.append(0.00)
            vertices_selecionados_final.append(vertices_str)
            continue
        
        txt_filename = f"{nome_base}.txt"
        txt_path = os.path.join(txt_folder, txt_filename)
        
        if not os.path.exists(txt_path):
            print(f"Arquivo .txt não encontrado para a instância '{instancia_completa}': {txt_path}")
            instancias_final.append(nome_base)
            modelos_final.append(modelo)
            diversidade_maxima_final.append(0.00)
            vertices_selecionados_final.append(vertices_str)
            continue
        
        try:
            # Ler o arquivo .txt como DataFrame
            # Ajuste o parâmetro 'sep' conforme o delimitador usado nos seus arquivos .txt
            # Aqui, assumo que o delimitador é espaço
            df_adjacencia = pd.read_csv(txt_path, sep=" ", header=None, skiprows=1, names=['V1', 'V2', 'Peso'])
            
            # Filtrar apenas as arestas onde ambos os vértices estão na lista selecionada
            df_filtrado = df_adjacencia[
                df_adjacencia['V1'].astype(str).isin(vertices) &
                df_adjacencia['V2'].astype(str).isin(vertices)
            ]
            
            # Calcular a soma dos pesos dessas arestas
            soma = df_filtrado['Peso'].sum()
            
            # Arredondar a soma para duas casas decimais
            soma_formatada = round(soma, 2)
            
            # Adicionar os resultados às listas finais
            instancias_final.append(nome_base)
            modelos_final.append(modelo)
            diversidade_maxima_final.append(soma_formatada)
            vertices_selecionados_final.append(vertices_str)
        
        except Exception as e:
            print(f"Erro ao processar o arquivo '{txt_filename}': {e}")
            instancias_final.append(nome_base)
            modelos_final.append(modelo)
            diversidade_maxima_final.append(0.00)
            vertices_selecionados_final.append(vertices_str)

    # Criar um novo DataFrame com os resultados
    df_final = pd.DataFrame({
        "Instância": instancias_final,
        "Modelo": modelos_final,
        "Valor da Diversidade Máxima": diversidade_maxima_final,
        "Vértices Selecionados": vertices_selecionados_final
    })

    # Formatar a coluna 'Valor da Diversidade Máxima' para ter exatamente duas casas decimais
    df_final["Valor da Diversidade Máxima"] = df_final["Valor da Diversidade Máxima"].map("{:.2f}".format)

    # Salvar o DataFrame final em um novo arquivo CSV
    df_final.to_csv(csv_output_path, index=False)

    print(f"Processamento concluído. Resultados salvos em '{csv_output_path}'.")
    print(df_final.head())

if __name__ == "__main__":
    main()


Processamento concluído. Resultados salvos em '../Main/Resultados Modelos Matriz/resultados_processados.csv'.
           Instância        Modelo Valor da Diversidade Máxima  \
0  MDG-a_15_n500_m50  RandomForest                     6132.81   
1  MDG-a_16_n500_m50  RandomForest                     6409.65   
2  MDG-a_17_n500_m50  RandomForest                     6460.24   
3  MDG-a_18_n500_m50  RandomForest                     6463.02   
4  MDG-a_19_n500_m50  RandomForest                     6224.35   

                               Vértices Selecionados  
0  [1, 6, 38, 43, 63, 67, 74, 75, 122, 125, 130, ...  
1  [1, 9, 15, 18, 42, 44, 51, 67, 71, 78, 97, 108...  
2  [17, 24, 30, 40, 46, 71, 82, 95, 122, 144, 148...  
3  [6, 63, 67, 68, 91, 93, 104, 106, 108, 113, 12...  
4  [7, 13, 29, 35, 39, 44, 53, 66, 72, 75, 79, 89...  


In [1]:
import os
import pandas as pd

def comparar_diversidade_maxima(csv_ideal_path, csv_resultados_path, csv_saida_path):
    """
    Compara os valores de Diversidade Máxima entre dois arquivos CSV e calcula a proximidade em porcentagem.

    Args:
        csv_ideal_path (str): Caminho para o arquivo CSV com valores ótimos.
        csv_resultados_path (str): Caminho para o arquivo CSV com valores obtidos.
        csv_saida_path (str): Caminho para salvar o arquivo CSV de saída.
    """
    
    # Verificar se os arquivos CSV de entrada existem
    if not os.path.exists(csv_ideal_path):
        raise FileNotFoundError(f"O arquivo CSV ideal '{csv_ideal_path}' não foi encontrado.")
    if not os.path.exists(csv_resultados_path):
        raise FileNotFoundError(f"O arquivo CSV de resultados '{csv_resultados_path}' não foi encontrado.")
    
    # Ler os arquivos CSV
    try:
        df_ideal = pd.read_csv(csv_ideal_path, sep=",")
    except Exception as e:
        raise ValueError(f"Erro ao ler o CSV ideal: {e}")
    
    try:
        df_resultados = pd.read_csv(csv_resultados_path, sep=",")
    except Exception as e:
        raise ValueError(f"Erro ao ler o CSV de resultados: {e}")
    
    # Remover espaços em branco dos nomes das colunas
    df_ideal.columns = df_ideal.columns.str.strip()
    df_resultados.columns = df_resultados.columns.str.strip()
    
    # Imprimir os nomes das colunas para depuração
    print("Colunas no CSV Ideal:")
    print(df_ideal.columns.tolist())
    print("\nColunas no CSV de Resultados:")
    print(df_resultados.columns.tolist())
    
    # Verificar se as colunas necessárias existem
    colunas_necessarias_ideal = ["Instância", "Valor da Diversidade Máxima"]
    for coluna in colunas_necessarias_ideal:
        if coluna not in df_ideal.columns:
            raise ValueError(f"A coluna '{coluna}' não foi encontrada no CSV ideal.")
    
    colunas_necessarias_resultados = ["Instância", "Modelo", "Valor da Diversidade Máxima", "Vértices Selecionados"]
    for coluna in colunas_necessarias_resultados:
        if coluna not in df_resultados.columns:
            raise ValueError(f"A coluna '{coluna}' não foi encontrada no CSV de resultados.")
    
    # Selecionar apenas as colunas necessárias
    df_ideal_subset = df_ideal[['Instância', 'Valor da Diversidade Máxima']].copy()
    df_resultados_subset = df_resultados[['Instância', 'Modelo', 'Valor da Diversidade Máxima', 'Vértices Selecionados']].copy()
    
    # Renomear as colunas para evitar confusão após o merge
    df_ideal_subset.rename(columns={'Valor da Diversidade Máxima': 'Valor_Otimo'}, inplace=True)
    df_resultados_subset.rename(columns={'Valor da Diversidade Máxima': 'Valor_Resultado'}, inplace=True)
    
    # Converter os valores para numéricos (caso estejam como strings)
    df_ideal_subset['Valor_Otimo'] = pd.to_numeric(df_ideal_subset['Valor_Otimo'], errors='coerce')
    df_resultados_subset['Valor_Resultado'] = pd.to_numeric(df_resultados_subset['Valor_Resultado'], errors='coerce')
    
    # Verificar se há valores NaN após a conversão
    if df_ideal_subset['Valor_Otimo'].isnull().any():
        print("Atenção: Existem valores NaN em 'Valor_Otimo'.")
    if df_resultados_subset['Valor_Resultado'].isnull().any():
        print("Atenção: Existem valores NaN em 'Valor_Resultado'.")
    
    # Imprimir uma amostra dos dados filtrados
    print("\nAmostra do CSV Ideal Filtrado:")
    print(df_ideal_subset.head())
    
    print("\nAmostra do CSV de Resultados Filtrado:")
    print(df_resultados_subset.head())
    
    # Merge dos DataFrames com base na coluna 'Instância'
    df_comparacao = pd.merge(df_ideal_subset, df_resultados_subset, on='Instância', how='inner')
    
    # Calcular a Proximidade (%) 
    # Fórmula: (Valor_Resultado / Valor_Otimo) * 100
    # Se Valor_Otimo for 0, definir Proximidade como 0 para evitar divisão por zero
    df_comparacao['Proximidade (%)'] = df_comparacao.apply(
        lambda row: (row['Valor_Resultado'] / row['Valor_Otimo'] * 100) if row['Valor_Otimo'] != 0 else 0,
        axis=1
    )
    
    # Arredondar os valores para duas casas decimais
    df_comparacao['Valor_Otimo'] = df_comparacao['Valor_Otimo'].round(2)
    df_comparacao['Valor_Resultado'] = df_comparacao['Valor_Resultado'].round(2)
    df_comparacao['Proximidade (%)'] = df_comparacao['Proximidade (%)'].round(2)
    
    # Reorganizar as colunas conforme solicitado
    df_final = df_comparacao[['Instância', 'Modelo', 'Valor_Otimo', 'Valor_Resultado', 'Proximidade (%)', 'Vértices Selecionados']].copy()
    
    # Salvar o DataFrame final em um novo arquivo CSV
    df_final.to_csv(csv_saida_path, index=False, float_format='%.2f')
    
    print(f"\nComparação concluída. Arquivo de saída salvo em '{csv_saida_path}'.")
    print("\nExemplo das primeiras linhas do arquivo de saída:")
    print(df_final.head())

if __name__ == "__main__":
    # Defina os caminhos para os arquivos CSV
    csv_ideal = "../Great values/great_values.csv"                     # Substitua pelo caminho real do seu CSV ideal
    csv_resultados = "../Main/Resultados Modelos Matriz/resultados_processados.csv"   # Substitua pelo caminho real do seu CSV de resultados
    csv_saida = "../Main/Resultados Modelos Matriz/comparacao_multiplos_modelos.csv"  # Nome do arquivo de saída
    
    # Chamar a função de comparação
    comparar_diversidade_maxima(csv_ideal, csv_resultados, csv_saida)


Colunas no CSV Ideal:
['Instância', 'Valor da Diversidade Máxima', 'Vértices Selecionados']

Colunas no CSV de Resultados:
['Instância', 'Modelo', 'Valor da Diversidade Máxima', 'Vértices Selecionados']

Amostra do CSV Ideal Filtrado:
         Instância  Valor_Otimo
0  GKD-a_10_n10_m3    435.71251
1  GKD-a_11_n10_m4    649.72168
2  GKD-a_12_n10_m4   1181.47302
3  GKD-a_13_n10_m4    733.43340
4  GKD-a_14_n10_m4    999.46989

Amostra do CSV de Resultados Filtrado:
                        Instância        Modelo  Valor_Resultado  \
0  GKD-a_5_n10_m2_caracteristicas  RandomForest              0.0   
1  GKD-a_6_n10_m3_caracteristicas  RandomForest              0.0   
2  GKD-a_7_n10_m3_caracteristicas  RandomForest              0.0   
3  GKD-a_8_n10_m3_caracteristicas  RandomForest              0.0   
4  GKD-a_9_n10_m3_caracteristicas  RandomForest              0.0   

  Vértices Selecionados  
0                [8, 9]  
1             [0, 7, 9]  
2             [2, 6, 9]  
3             [2, 3,

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurações de estilo para os gráficos
sns.set(style="whitegrid")
plt.rcParams.update({'figure.max_open_warning': 0})

def gerar_graficos(csv_path, output_folder):
    """
    Gera gráficos de visualização a partir do CSV de comparação.

    Args:
        csv_path (str): Caminho para o arquivo CSV de comparação.
        output_folder (str): Pasta onde os gráficos serão salvos.
    """

    # Ler o arquivo CSV
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        raise ValueError(f"Erro ao ler o CSV de comparação: {e}")

    # Verificar se as colunas necessárias existem
    colunas_necessarias = ["Instância", "Modelo", "Valor_Otimo", "Valor_Resultado", "Proximidade (%)", "Vértices Selecionados"]
    for coluna in colunas_necessarias:
        if coluna not in df.columns:
            raise ValueError(f"A coluna '{coluna}' não foi encontrada no CSV de comparação.")

    # Criar a pasta de saída se não existir
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 1. Gráfico de Barras por Instância
    instancias = df['Instância'].unique()
    for instancia in instancias:
        df_instancia = df[df['Instância'] == instancia]
        modelos = df_instancia['Modelo']
        valores_otimos = df_instancia['Valor_Otimo']
        valores_resultados = df_instancia['Valor_Resultado']

        x = range(len(modelos))

        plt.figure(figsize=(10, 6))
        plt.bar(x, valores_otimos, width=0.4, label='Valor Ótimo', align='center')
        plt.bar(x, valores_resultados, width=0.4, label='Valor Resultado', align='edge')
        plt.xlabel('Modelo')
        plt.ylabel('Valor da Diversidade Máxima')
        plt.title(f'Comparação de Diversidade Máxima para {instancia}')
        plt.xticks(x, modelos, rotation=45)
        plt.legend()
        plt.tight_layout()

        # Salvar o gráfico
        plt.savefig(os.path.join(output_folder, f'barra_{instancia}.png'))
        plt.close()

    print(f"Gráficos de barras por instância salvos em '{output_folder}'.")

    # 2. Gráfico de Proximidade (%) por Modelo
    plt.figure(figsize=(12, 8))
    sns.barplot(
        x='Modelo',
        y='Proximidade (%)',
        data=df,
        errorbar=None,  # Substitui ci=None
        palette='viridis',
        estimator=lambda x: sum(x)/len(x)
    )
    plt.xlabel('Modelo')
    plt.ylabel('Proximidade Média (%)')
    plt.title('Proximidade Média (%) por Modelo')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'proximidade_media_por_modelo.png'))
    plt.close()

    print(f"Gráfico de proximidade média por modelo salvo em '{output_folder}'.")

    # 3. Boxplot de Proximidade (%) por Modelo
    plt.figure(figsize=(12, 8))
    sns.boxplot(
        x='Modelo',
        y='Proximidade (%)',
        data=df,
        hue='Modelo',  # Adiciona hue para evitar o FutureWarning
        palette='Set2'
    )
    plt.xlabel('Modelo')
    plt.ylabel('Proximidade (%)')
    plt.title('Distribuição da Proximidade (%) por Modelo')
    plt.xticks(rotation=45)
    plt.legend([],[], frameon=False)  # Remove a legenda duplicada
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'boxplot_proximidade_por_modelo.png'))
    plt.close()

    print(f"Boxplot de proximidade por modelo salvo em '{output_folder}'.")

    # 4. Heatmap de Proximidade (%) por Instância e Modelo
    heatmap_data = df.pivot(index="Instância", columns="Modelo", values="Proximidade (%)")
    plt.figure(figsize=(15, 10))
    sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="YlGnBu")
    plt.xlabel('Modelo')
    plt.ylabel('Instância')
    plt.title('Heatmap da Proximidade (%) por Instância e Modelo')
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'heatmap_proximidade.png'))
    plt.close()

    print(f"Heatmap de proximidade salvo em '{output_folder}'.")

    # 5. Gráfico de Linha de Proximidade por Modelo
    plt.figure(figsize=(15, 10))
    for modelo in df['Modelo'].unique():
        df_modelo = df[df['Modelo'] == modelo]
        plt.plot(df_modelo['Instância'], df_modelo['Proximidade (%)'], marker='o', label=modelo)
    
    plt.xlabel('Instância')
    plt.ylabel('Proximidade (%)')
    plt.title('Proximidade (%) por Instância e Modelo')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'linha_proximidade_por_modelo.png'))
    plt.close()

    print(f"Gráfico de linha de proximidade por modelo salvo em '{output_folder}'.")

    # 6. Scatter Plot: Valor Ótimo vs Valor Resultado, colorido por Modelo
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x='Valor_Otimo', y='Valor_Resultado', hue='Modelo', data=df, palette='deep')
    plt.plot(
        [df['Valor_Otimo'].min(), df['Valor_Otimo'].max()],
        [df['Valor_Otimo'].min(), df['Valor_Otimo'].max()],
        'k--', label='Linha de Igualdade'
    )
    plt.xlabel('Valor Ótimo')
    plt.ylabel('Valor Resultado')
    plt.title('Valor Resultado vs Valor Ótimo por Modelo')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'scatter_valor_vs_otimo.png'))
    plt.close()

    print(f"Scatter plot de Valor Ótimo vs Valor Resultado salvo em '{output_folder}'.")

    # 7. Gráfico de Distribuição de Proximidade (%) para Todos os Modelos
    plt.figure(figsize=(12, 8))
    sns.histplot(data=df, x='Proximidade (%)', hue='Modelo', kde=True, multiple='stack', palette='bright')
    plt.xlabel('Proximidade (%)')
    plt.ylabel('Contagem')
    plt.title('Distribuição da Proximidade (%) por Modelo')
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'distribuicao_proximidade_por_modelo.png'))
    plt.close()

    print(f"Histograma de distribuição de proximidade por modelo salvo em '{output_folder}'.")

if __name__ == "__main__":
    # Defina o caminho para o CSV gerado anteriormente
    csv_comparacao = "../Main/Resultados Modelos Matriz/comparacao_multiplos_modelos.csv"  # Substitua pelo caminho correto, se necessário
    
    # Defina a pasta onde os gráficos serão salvos
    pasta_saida = "Main/Resultados Modelos Matriz"

    # Chamar a função para gerar os gráficos
    gerar_graficos(csv_comparacao, pasta_saida)


Gráficos de barras por instância salvos em 'graficos_resultados'.
Gráfico de proximidade média por modelo salvo em 'graficos_resultados'.


C:\Users\vinic\AppData\Local\Temp\ipykernel_8108\659334930.py:63: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


Boxplot de proximidade por modelo salvo em 'graficos_resultados'.
Heatmap de proximidade salvo em 'graficos_resultados'.
Gráfico de linha de proximidade por modelo salvo em 'graficos_resultados'.
Scatter plot de Valor Ótimo vs Valor Resultado salvo em 'graficos_resultados'.
Histograma de distribuição de proximidade por modelo salvo em 'graficos_resultados'.
